In [1]:
import getpass
import os

os.environ["OPENAI_API_KEY"] = getpass.getpass()

··········


In [3]:
!pip install langchain_openai langchain_core langchain

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.4/50.4 kB 2.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 52.0/52.0 kB 4.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 393.9/393.9 kB 27.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 997.8/997.8 kB 46.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 148.9/148.9 kB 10.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 362.9/362.9 kB 19.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 41.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.6/75.6 kB 5.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.9/77.9 kB 5.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 318.9/318.9 kB 18.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 141.9/141.9 kB 10.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 3.7 MB/s eta 0:00:00
  Attempting uninstall: tenacity
    Found ex

In [4]:
from langchain_openai import ChatOpenAI

llm = ChatOpenAI(model="gpt-4o-mini", temperature=0)

In [14]:
import random
from langchain.tools import BaseTool, StructuredTool, tool

@tool
def weather(query: str) -> str:
    """Procura pelo clima de uma cidade online"""
    weather = ["ensolarado", "nublado", "chuvoso", "neve"]
    result = random.choice(weather)
    return result

@tool
def multiply(a: int, b: int) -> int:
    """Multiplica dois números"""
    return a * b

@tool
def write_file(file_name: str, content: str) -> None:
    """Escreve um arquivo de texto"""
    with open(file_name, "w") as f:
        f.write(content)

@tool
def count_letters(word: str, letter: str) -> int:
    """Conta a quantidade de letras de uma palavra"""
    return word.count(letter)

tools = [weather, multiply, write_file, count_letters]
llm_with_tools = llm.bind_tools(tools)

In [15]:
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder

prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            "Você é um assistente mas não sabe do clima ou fazer cálculos matemáticos.",
        ),
        ("user", "{input}"),
        MessagesPlaceholder(variable_name="agent_scratchpad"),
    ]
)

In [16]:
from langchain.agents.format_scratchpad.openai_tools import format_to_openai_tool_messages
from langchain.agents.output_parsers.openai_tools import OpenAIToolsAgentOutputParser
from langchain.agents import AgentExecutor

agent = (
    {
        "input": lambda x: x["input"],
        "agent_scratchpad": lambda x: format_to_openai_tool_messages(
            x["intermediate_steps"]
        ),
    }
    | prompt
    | llm_with_tools
    | OpenAIToolsAgentOutputParser()
)

agent_executor = AgentExecutor(agent=agent, tools=tools, verbose=True)

In [8]:
agent_executor.invoke({"input": "Qual o clima em Recife?"})



> Entering new AgentExecutor chain...

Invoking: `weather` with `{'query': 'Recife'}`


ensolaradoO clima em Recife está ensolarado.

> Finished chain.


{'input': 'Qual o clima em Recife?',
 'output': 'O clima em Recife está ensolarado.'}

## Exercícios

### Exercício 1
Adicione uma função nas ferramentas do agente para criar arquivos de texto. Em seguida, teste o agente.

In [13]:
agent_executor.invoke({"input": "Crie um arquivo de texto chamado 'exemplo.txt' contendo informações sobre o clima de Garanhuns'."})



> Entering new AgentExecutor chain...

Invoking: `weather` with `{'query': 'Garanhuns'}`


ensolarado
Invoking: `write_file` with `{'file_name': 'exemplo.txt', 'content': 'Informações sobre o clima de Garanhuns:\n\nO clima atual é ensolarado.'}`


NoneO arquivo 'exemplo.txt' foi criado com as informações sobre o clima de Garanhuns, que está ensolarado. Se precisar de mais alguma coisa, é só avisar!

> Finished chain.


{'input': "Crie um arquivo de texto chamado 'exemplo.txt' contendo informações sobre o clima de Garanhuns'.",
 'output': "O arquivo 'exemplo.txt' foi criado com as informações sobre o clima de Garanhuns, que está ensolarado. Se precisar de mais alguma coisa, é só avisar!"}

### Exercício 2
Crie um agente que conte a quantidade de ocorrências de uma determinada letra em uma palavra. Em seguida, teste o agente.

In [17]:
agent_executor.invoke({"input": "Conte a quantidade de letras 'p' na palavra 'paralelepípedo'."})



> Entering new AgentExecutor chain...

Invoking: `count_letters` with `{'word': 'paralelepípedo', 'letter': 'p'}`


3A palavra "paralelepípedo" contém 3 letras 'p'.

> Finished chain.


{'input': "Conte a quantidade de letras 'p' na palavra 'paralelepípedo'.",
 'output': 'A palavra "paralelepípedo" contém 3 letras \'p\'.'}